# Import Items

### Import Libraries

In [10]:
import glob
import os
import json
import xml.etree.ElementTree as ET
import requests
import urllib.request
import time
import configparser
import tqdm

### Get configuration file

In [11]:
config = configparser.ConfigParser()
config.sections()
config.read('../config/api.ini')
client_id = config.get('main', 'client_id')
client_secret = config.get('main', 'client_secret')
endpoint = config.get('main', 'endpoint')

### Connect to the Api

In [12]:
params = {
    'key_identity': client_id,
    'key_credential': client_secret
}

access_url = endpoint
print("Connected to:    ", access_url)

Connected to:     http://localhost:8080/api/


### Navigate the xml and converts it to json

In [19]:
dictionary = {}
fileName = ''

for file in glob.glob("../input/*"):
    root = ET.parse(file).getroot()
    fileName = str(os.path.basename(file))
    
    for group in root.findall('family/group'):
        groupName = group.attrib['name']
        item = group.findall('item')
        
        for value in item:
            fieldName = value.attrib['name']         
            matchingName = groupName +"_"+ fieldName
            fieldValue = value.text
            
            #fix getty links 
            if "getty" in str(fieldValue) and 'subjectid=' in str(fieldValue) and '[' in str(fieldValue) and '[' in str(fieldValue):
                name  = fieldValue.split('[')[0].split(']')[0]
                url  = fieldValue.split('[')[1].split(']')[0]

                if  'subjectid=' in str(url):
                    url  = url.split('subjectid=')[1]
                    fieldValue = name + "; http://vocab.getty.edu/page/tgn/" +  url            

            cleanedText = str(fieldValue).replace("None", "").replace("\n", "").replace("\t", " ").replace('"', '').replace("'", "")
            
            # Creating/appending to a JSON structure
            if fileName not in dictionary:
                dictionary[fileName] = { matchingName :  [cleanedText] }
            else:
                if matchingName not in dictionary[fileName]:
                    dictionary[fileName][matchingName] = [cleanedText]
                else:
                    dictionary[fileName][matchingName].append(cleanedText)

### Define basic omeka items

In [16]:
omekaTemplate =  { "o:resource_template": {"@id": endpoint + "/resource_templates\/2","o:id": 2}}
omekaClass = {"o:resource_class": {"o:id": 592}}

### Convert Data Fields

In [18]:
for i in tqdm.tqdm(dictionary.keys()):
    
    # Converting entries
    identifier = { "crm:P1_is_identified_by": [ { "type": "literal", "property_id": 1899, "@value": dictionary[i]['Objekt_Sammlungsobjekt-ID'][0] } ]}

    postFields = json.dumps({ **omekaClass, **omekaTemplate, **identifier})
    
    # Post Request
    url = access_url + 'items/'
    headers = {"Content-Type": "application/json"}
    r = requests.post(url, data=postFields, params = params, headers=headers)
    time.sleep(.25)

100%|██████████| 641/641 [09:57<00:00,  1.07it/s]
